<a href="https://colab.research.google.com/github/JonathanLehner/Colab-collection/blob/master/MRCNN_counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mask R-CNN Demo - Counting

This is a [Mask R-CNN](https://arxiv.org/abs/1703.06870) colab notebook using the open source project [matterport/Mask_RCNN](https://github.com/matterport/Mask_RCNN).

Follow this to make your picam stream:
https://docs.dataplicity.com/docs/stream-live-video-from-your-pi

Our pi is at 
https://coverless-gnu-0291.dataplicity.io/?action=stream

In [0]:
# define variables
import os
repository_url = 'https://github.com/JonathanLehner/Colab-collection'
repository_name = os.path.basename(repository_url)
loader_path = './products.py'
# only check huerlimann
loader_path = './huerlimann.py'

products_model = 'https://github.com/JonathanLehner/Colab-collection/releases/download/0.1/mask_rcnn_products_0120_new.h5'
# only check huerlimann
products_model = 'https://github.com/JonathanLehner/Colab-collection/releases/download/0.1/mask_rcnn_huerlimann_0060.h5'
products_model_name = os.path.basename(products_model)
products_data_url = 'https://github.com/JonathanLehner/Colab-collection/releases/download/0.1/labeled_training_images3.0.zip'
products_filename = os.path.basename(products_data_url)
products_filename_wo_ext = 'labeled_training_images'

server_address = 'www.grabigo.com'
pi_address = 'https://coverless-gnu-0291.dataplicity.io/?action=stream'
test_category = "/test_do_not_label"

# Products class names
# Index of the class in the list is its ID. For example, to get ID of
# the huerlimann class, use: class_names.index('huerlimann')
class_names = ['BG', "apple", "banana", "feldschloesschen", "huerlimann", "ramen", "redbull", "tuc", "tuna", "x", "y", "z"]
# only check huerlimann
class_names = ['BG', "huerlimann"]

mode = "counting"

SHELF_ID = "BKc4cY4DBw65XcTpc"

In [0]:
import os
from os.path import exists, join, basename

project_name = "Mask_RCNN"
if not exists(project_name):
  # clone and install
  !git clone -q https://github.com/matterport/Mask_RCNN.git
  !cd $project_name && pip install -q -r requirements.txt
  
import sys
sys.path.append(project_name)

In [30]:
# download model files
%cd /content/Mask_RCNN/
os.system('rm {0}'.format(products_model_name))
os.system('wget {0}'.format(products_model))
%cd /content/

/content/Mask_RCNN
/content


In [31]:
# download training dataset
os.system('wget {0}'.format(products_data_url))
print('unzip {0}'.format(products_filename))
os.system('unzip {0}'.format(products_filename))

!rm -rf /content/Mask_RCNN/datasets
!mkdir /content/Mask_RCNN/datasets

print('mv /content/{0} /content/Mask_RCNN/datasets/{1}'.format(products_filename_wo_ext,products_filename_wo_ext))
os.system('mv /content/{0} /content/Mask_RCNN/datasets/{1}'.format(products_filename_wo_ext,products_filename_wo_ext))

unzip labeled_training_images3.0.zip
mv /content/labeled_training_images /content/Mask_RCNN/datasets/labeled_training_images


0

In [32]:
%cd /content/Mask_RCNN
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = os.path.abspath("/content/Mask_RCNN/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to Products trained weights
# You can download this file from the Releases page
# https://github.com/matterport/Mask_RCNN/releases
PRODUCTS_WEIGHTS_PATH = "/content/Mask_RCNN/{0}".format(products_model_name)  # TODO: update this path

/content/Mask_RCNN


## Configurations

In [33]:
# load the training script
%cd /content/Mask_RCNN/samples/
os.system('rm -r {0}'.format(repository_name))

print('git clone {0}'.format(repository_url))
os.system('git clone {0}'.format(repository_url))

/content/Mask_RCNN/samples
git clone https://github.com/JonathanLehner/Colab-collection


0

In [34]:
%cd /content/Mask_RCNN/samples/Colab-collection
# products = __import__("products")
products = __import__("huerlimann")
%cd /content/Mask_RCNN/

config = products.ProductConfig()
PRODUCTS_DIR = os.path.join(ROOT_DIR, "datasets/{0}".format(products_filename_wo_ext))

/content/Mask_RCNN/samples/Colab-collection
/content/Mask_RCNN


In [35]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Notebook Preferences

In [0]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [0]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax



## Load Validation Dataset

In [38]:
# Load validation dataset
dataset = products.ProductDataset()

print(products)
dataset.load_products(PRODUCTS_DIR+test_category, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

<module 'huerlimann' from '/content/Mask_RCNN/samples/Colab-collection/huerlimann.py'>
Images: 67
Classes: ['BG', 'huerlimann']


## Load Model

In [0]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [40]:
# Set path to products weights file

# Download file from the Releases page and set its path
# https://github.com/matterport/Mask_RCNN/releases
weights_path = '/content/Mask_RCNN/{0}'.format(products_model_name)

# Or, load the last model you trained
# weights_path = model.find_last()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  /content/Mask_RCNN/mask_rcnn_huerlimann_0060.h5


## Set up the connection to Meteor server

In [49]:
# Connect to the server
!pip install python-meteor
from MeteorClient import MeteorClient
client = MeteorClient('wss://{0}/websocket'.format(server_address))
client.connect()
print('connected')

connected


In [0]:
# define callback functions
def changed(collection, id, fields, cleared):
    print('* CHANGED {} {}'.format(collection, id))
    if(collection == "users"):
        for key, value in fields.items():
            if(key == "profile"):
                profile = value
                #print(profile)
                basket_content = profile['basket_content']
                print(basket_content)
                for item in basket_content:
                    item_label = item['name']
                    item_qty = item['quantity']
                    products[item_label] = item_qty
    for key, value in fields.items():
        print('  - FIELD {} {}'.format(key, value))
    for key, value in cleared.items():
        print('  - CLEARED {} {}'.format(key, value))
        
client.on('changed', changed)

def callback_function(error, result):
    print('callback')
    print(result)
    if error:
        print(errofr)
        return

    print(result)

## Run Detections on stream from Raspberry Pi

In [51]:
import time
import cv2
import urllib.request as ur
import numpy as np
from google.colab.patches import cv2_imshow
import collections


# run sudo ifconfig inside the pi to get the local network address "inet" but then need local hw 
# http://10.4.14.67/?action=stream
# https://stackoverflow.com/questions/54949143/opencv-unable-to-read-video-from-mjpeg-stream
stream=ur.urlopen(pi_address)
bytestring=b''
counter = 0
counter_list_prev = dict()
while True:
    bytestring+=stream.read(1024)
    a = bytestring.find(b'\xff\xd8')
    b = bytestring.find(b'\xff\xd9')
    if a !=- 1 and b != -1:
        jpg = bytestring[a:b+2]
        bytestring= bytestring[b+2:]
        i = cv2.imdecode(np.fromstring(jpg, dtype=np.uint8),cv2.IMREAD_COLOR)
        
        if(mode == "training" and counter % 10 == 0): # every second
            #save image to file
            img_label = training_label + str(counter) + ".jpg"
            cv2.imwrite(os.path.join('training_images/'+training_label, img_label), i)
            cv2_imshow(i)

        if(mode == "counting" and counter % 30 == 0): # with 10FPS this is one per three seconds
            #from matplotlib import pyplot as plt
            #plt.subplot(211),plt.imshow(i)
            #plt.xticks([]),plt.yticks([])
            #plt.pause(0.2);
            #cv2_imshow(i)
          
            image = cv2.cvtColor(i,cv2.COLOR_BGR2RGB)
            # Run detection
            results = model.detect([image], verbose=0)
            #results = model.detect([image], verbose=1)
            #print(results)

            # Visualize results
            r = results[0]
            visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                class_names, r['scores'])

            class_ids = r['class_ids']
            counter_list = collections.Counter(class_ids)
            # check if hand is in counter_list
            # if yes, then do nothing --> to prevent too frequent changes
            # in virtual basket when customer is just looking at products
             
            for key in counter_list:
                if not key in counter_list_prev:
                    counter_list_prev[key] = 0

                print("Object: {}, Quantity: {}".format(class_names[key], counter_list[key]))
                if counter_list_prev[key] != counter_list[key]:
                    difference = counter_list[key] - counter_list_prev[key]
                    print(" -- Quantity changed by {}".format(difference))
                    #client.call('modifyBasket', [SHELF_ID, int(key), difference], callback_function)

            for key in counter_list_prev:  
                if key in counter_list_prev and key not in counter_list:
                  print(" -- Object: {}, Quantity: {} was removed".format(class_names[key], counter_list_prev[key]))
                  #client.call('modifyBasket', [SHELF_ID, int(key), -counter_list_prev[key]], callback_function)

            #save detection result for comparison
            counter_list_prev = counter_list

        counter += 1
        print(counter)
        
    #not meaningful inside Google colab
    #if cv2.waitKey(1) & 0xFF == ord('q'):
    #    break

    #time.sleep(0.1)

timeout: ignored